# Jeremy's Notebook

# Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import sirtipy
import pdb
import itertools
import time
import os
from scipy.integrate import trapz, quad
from scipy.special import gamma as gammafunc
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

# Load Data

In [2]:
# ISOCAM
infile1   = 'lutz.dat'
lutzdata  = np.loadtxt(infile1)
lutz_freq = lutzdata[:,0]
lutz_flux = lutzdata[:,1]

infile2     = 'draine.dat'
drainedata  = np.loadtxt(infile2)
draine_freq = drainedata[:,0]
draine_flux = drainedata[:,1]

# GBI
infile3  = 'gbi.dat'
gbidata  = np.loadtxt(infile3)
gbi_freq = gbidata[:,0]
gbi_flux = gbidata[:,1]

## Scale Data and Calculate Error

In [403]:
# Constants
r_sun    = const.R_sun.cgs.value
r_disk   = 1e12
distance = (11.2 * u.kpc).cgs.value

# Frequency and flux scale factor
scale_freq = 1e9
scale_flux = 1e-26 / (((np.pi * 23.5 * r_sun) / distance)**2)

# Calculate the error for each frequency point
# Range found in Fuchs, Mirabel, Claret 2018
error_lw4 = np.abs(4.61  - 5.45) * 1e12
error_lw6 = np.abs(3.53  - 4.28) * 1e12
error_lw7 = np.abs(2.60  - 3.53) * 1e12
error_lw8 = np.abs(2.50  - 2.8)  * 1e12
error_lw9 = np.abs(1.875 - 2.14) * 1e12
error_lw3 = np.abs(1.67  - 2.5)  * 1e12

# Set upper and lower limits for frequency
xlolims = np.array([4.61, 3.53, 2.6, 2.5, 1.875, 1.67]) * 1e13
xuplims = np.array([5.45, 4.28, 3.53, 2.8, 2.14, 2.5]) * 1e13

# ISOCAM
lutz_freq_scaled = lutz_freq * scale_freq
lutz_flux_scaled = lutz_flux * scale_flux
lutz_error_freq  = np.array([error_lw9, error_lw3, error_lw8, error_lw7, error_lw6, error_lw4])
lutz_error_flux  = np.array([7.2, 5.5, 5.8, 7.0, 12., 11.])

draine_freq_scaled = draine_freq * scale_freq
draine_flux_scaled = draine_flux * scale_flux
draine_error_freq  = np.array([error_lw9, error_lw3, error_lw8, error_lw7, error_lw6, error_lw4])
draine_error_flux  = np.array([7.2, 5.5, 6.0, 6.9, 5.9, 4.8])

# GBI data 
gbi_freq_scaled = gbi_freq * scale_freq
gbi_flux_scaled = gbi_flux * scale_flux
gbi_error_freq  = np.array([4., 6.]) * scale_freq
gbi_error_flux  = np.array([4., 6.])
crit_freq       = gbi_freq_scaled * .1
crit_flux       = gbi_flux * .1

# Helper Functions

In [267]:
# Useful functions

# Figure out the normalization constant C such that
#  n = C int_gmin^gmax g^-p dg
def powerlaw_normalize(p, gmin, gmax, ne):
    integral = (gmax**(1. - p) - gmin**(1. - p)) / (1. - p)
    return ne / integral

# Synchrotron emission coefficient from a power-law electron distribution.
# Based on RL equation 6.36.
def j_synchrotron(frequency, location, Inu, B, s_1, s_2, gmin, gmax, ne, cutoff_freq):
    # Parameters: B = magnetic field (Gauss),  p = power law index
    # gmin, gmax = gamma range for power law, ne = electron number density (cm^-3)
    # cutoff_freq = frequency range for two synchrotrons (False by default for no cutoff)
    p_1 = (2. * s_1) + 1.
    p_2 = (2. * s_2) + 1.
    
    # Figure out normalization C
    C_1 = powerlaw_normalize(p_1, gmin, gmax, ne)
    C_2 = powerlaw_normalize(p_2, gmin, gmax, ne)
    
    sinalpha = 0.91
    
    # RL 6.36. Note that in RL it is per unit omega, so we need to multiply by 2pi (and also
    # use nu instead of omega in the formula)
    Ptot_1 = np.sqrt(3.) * const.e.esu.value**3 * C_1 * B * sinalpha / (const.m_e.cgs.value * const.c.cgs.value**2 *
        (p_1 + 1.)) * gammafunc((0.25 * p_1) + (19. / 12.)) * gammafunc((0.25 * p_1) - (1. / 12.)) * (const.m_e.cgs.value * const.c.cgs.value *
        frequency * 2. * np.pi / (3. * const.e.esu.value * B * sinalpha))**(-(p_1 - 1.) / 2.)
    
    Ptot_2 = np.sqrt(3.) * const.e.esu.value**3 * C_2 * B * sinalpha / (const.m_e.cgs.value * const.c.cgs.value**2 *
        (p_2 + 1.)) * gammafunc((0.25 * p_2) + (19. / 12.)) * gammafunc((0.25 * p_2) - (1. / 12.)) * (const.m_e.cgs.value * const.c.cgs.value *
        frequency * 2. * np.pi / (3. * const.e.esu.value * B * sinalpha))**(-(p_2 - 1.) / 2.)
    
    if cutoff_freq   == False:
        jnu = Ptot_1 / (4. * np.pi)   
    else:
        jnu = (((frequency < cutoff_freq) * Ptot_1) / (4. * np.pi)) + (((frequency > cutoff_freq) * Ptot_2) / (4. * np.pi))
    
    return jnu

def alpha_thermal(frequency, location, Inu, alphaval):
    return alphaval

def j_thermal(frequency, location, Inu, temperature, alphafunc, alphafunc_args):
    Bnu = sirtipy.blackbody_intensity(temperature, frequency)   
    emi = alphafunc(frequency, location, Inu, *alphafunc_args)*Bnu 
    return emi

# Converts intensity in cgs to flux density in milliJanksys
def inten_to_flux(intensity, radius):
    # 1 Jansky is 10^−23 erg⋅s−1⋅cm−2⋅Hz−1
    # 1 milliJansky is 10^-26 erg⋅s−1⋅cm−2⋅Hz−1
    flux_mJy = np.multiply(intensity, 1e26 * (np.pi * (radius / distance)**2))
    return flux_mJy

# Main Code

In [498]:
# Physical parameters for medium
gmin    = 2.
s_radio = 0.08 # From Fuchs
s_ir    = 0.8  # From Fuchs

# Create an empty initial logarithmically-spaced spectrum going from 1 GHz to 5e5 GHz
freqaxis                = sirtipy.frequency_axis('log', frange=[1*u.GHz, 5e5*u.GHz], numpts=500)
inspec_empty_sync       = sirtipy.spectrum('empty', frequencies=freqaxis)
inspec_empty_sync_ir    = sirtipy.spectrum('empty', frequencies=freqaxis)
inspec_empty_bb_stellar = sirtipy.spectrum('empty', frequencies=freqaxis)
inspec_empty_bb_ir      = sirtipy.spectrum('empty', frequencies=freqaxis)

# Stellar blackbody parameters (CONSTANT ALWAYS PRESENT)
alpha_bb_stellar       = 1e-10
medium_bb_stellar_temp = 4300.
radius_bb_stellar      = 23.5 * r_sun

# Transfer through a medium of depth 1e12 cm in steps of desired size.
# 1e12 is the max taken from Fuchs paper
s  = [0., 1e12]
ds = 5e8

# IR blackbody parameters (Variable)
alpha_bb_ir              = 1e-10
medium_bb_ir_temperature = [500, 900, 950, 1100]
medium_bb_ir_radius      = np.array([150, 330, 430, 850]) * r_sun

# Plot single or multiple distances (use np.geomspace() or np.linspace())
distances_sync_radio = np.geomspace(1e9, 1e12, 2000)
# These distances fixed due to inclination
distances_sync_ir    = [8e11]
distances_bb_stellar = [8e11]
distances_bb_ir      = [8e11]

# Initialize combined full combined spec array
# Will populate with parameters tested, best fit spectra,
# best fit residuals for GBI and ISOCAM, and sum of best fit residuals
full_spec_combo = [[],[],[],[]]

# Adjustable parameters for synchrotron: B, gmax, ne, and frequency cutoff
# Use np.geomspace() or np.linspace()
B    = np.geomspace(1e3, 1e6, 20)
gmax = np.geomspace(1e4, 1e7, 20)
ne   = np.geomspace(1e3, 1e6, 20)

# Choose different systems to simulate or plot
sim_sync_ir = True
sim_bb_ir   = False
dereddening = True

# Choose different IR blackbody radius and temperature
# All false if no IR blackbody desired
ir_500_850_bool  = False
ir_900_430_bool  = False
ir_950_330_bool  = False
ir_1100_150_bool = False

# Choose different cutoff frequencies for the radio/ir switch
# All false if no IR synchrotron cutoff desired
cutoff_frequency    = [2000e9, 9000e9, 31000e9]
cutoff_2000e9_bool  = False
cutoff_9000e9_bool  = False
cutoff_31000e9_bool = True

# Choose Lutz or Draine dereddening
dereddening_lutz   = False
dereddening_draine = True

# Choose to plot after each simulation or at the end
# True for individual and False for at the end
plot_individual = False

# Choose to save figure
save_fig = True

# Used to track for loop progression (purely for my own sanity)
counter     = 0                            # Keep track of iteration number
timer       = True                         # T/F print iteration runtime
start       = time.perf_counter()          # Start timer
end         = False                        # End timer
iterations  = len(B) * len(gmax) * len(ne) # Total number of iterations to run

# For loops to iterate through desired parameters
for i, j, k in itertools.product(B, gmax, ne):
    
    # Initialize counter
    counter += 1
    print(f'Iteration {counter} of {iterations}')
    
    # Print that program has started and with what parameters
    print(f'Begin B = {i:.3E}, gmax = {j:.3E}, ne = {k:.3E}')
    time_iteration = time.perf_counter()
    
    # Check critical booleans before starting
    if counter == 1:
        # Cutoff frequency for second synchrotron
        if sim_sync_ir   == False:
            cutoff_freq  = False
            print('No secondary synchrotron')
        elif (cutoff_2000e9_bool  == True) & (all([cutoff_9000e9_bool, cutoff_31000e9_bool]) == False):
            cutoff_freq  = cutoff_frequency[0]
            cutoff_index = np.where(np.isclose(freqaxis.frequencies, cutoff_freq, atol=cutoff_freq*.01))[0]
            print('IR synchrotron frequency cutoff 2000e9 GHz')
        elif (cutoff_9000e9_bool  == True) & (all([cutoff_2000e9_bool, cutoff_31000e9_bool]) == False):
            cutoff_freq  = cutoff_frequency[1]
            cutoff_index = np.where(np.isclose(freqaxis.frequencies, cutoff_freq, atol=cutoff_freq*.01))[0]
            print('IR synchrotron frequency cutoff 9000e9 GHz')
        elif (cutoff_31000e9_bool == True) & (all([cutoff_2000e9_bool, cutoff_9000e9_bool])  == False):
            cutoff_freq  = cutoff_frequency[2]
            cutoff_index = np.where(np.isclose(freqaxis.frequencies, cutoff_freq, atol=cutoff_freq*.01))[0]
            print('IR synchrotron frequency cutoff 31000e9 GHz')
        else:
            end = True
            print('Not valid value for cutoff frequencies for the radio/ir switch!')
            break

        # IR blackbody radius and temperature
        if sim_bb_ir == False:
            sim_bb_ir_name = 'No IR blackbody'
            print('No IR Blackbody')
        elif (ir_500_850_bool  == True) & (any([ir_900_430_bool, ir_950_330_bool, ir_1100_150_bool]) == False):
            medium_bb_ir_temp  = medium_bb_ir_temperature[0]
            medium_bb_ir_rad   = medium_bb_ir_radius[3]
            sim_bb_ir_name = f'IR blackbody {medium_bb_ir_temp} K and {medium_bb_ir_rad / r_sun} R$_\\odot$'
        elif (ir_900_430_bool  == True) & (any([ir_500_850_bool, ir_950_330_bool, ir_1100_150_bool]) == False):
            medium_bb_ir_temp  = medium_bb_ir_temperature[1]
            medium_bb_ir_rad   = medium_bb_ir_radius[2]
            sim_bb_ir_name = f'IR blackbody {medium_bb_ir_temp} K and {medium_bb_ir_rad / r_sun} R$_\\odot$'
        elif (ir_950_330_bool  == True) & (any([ir_500_850_bool, ir_900_430_bool, ir_1100_150_bool]) == False):
            medium_bb_ir_temp  = medium_bb_ir_temperature[2]
            medium_bb_ir_rad   = medium_bb_ir_radius[1]
            sim_bb_ir_name = f'IR blackbody {medium_bb_ir_temp} K and {medium_bb_ir_rad / r_sun} R$_\\odot$'
        elif (ir_1100_150_bool == True) & (any([ir_500_850_bool, ir_900_430_bool, ir_950_330_bool])  == False):
            medium_bb_ir_temp  = medium_bb_ir_temperature[3]
            medium_bb_ir_rad   = medium_bb_ir_radius[0]
            sim_bb_ir_name = f'IR blackbody {medium_bb_ir_temp} K and {medium_bb_ir_rad / r_sun:.3E} R$_\\odot$'
        else:
            end = True
            print('Not valid value for IR blackbody radius and temperature!')
            break
                                
        # Lutz or Draine dereddening
        if dereddening == False:
            plot  = 'No Dereddening'
            title = 'GRS 1915+105\n'
            print('No dereddening')
        elif (dereddening_lutz   == True) & (dereddening_draine == False):
            plot  = 'Lutz'
            title = 'GRS 1915+105 (Lutz Dereddening)\n'
            label = 'ISOCAM Data (Lutz Dereddening)'
            data  = lutzdata
            print('Lutz Dereddening')
        elif (dereddening_draine == True) & (dereddening_lutz   == False):
            plot  = 'Draine'
            title = 'GRS 1915+105 (Draine Dereddening)\n'
            label = 'ISOCAM Data (Draine Dereddening)'
            data  = drainedata
            print('Draine Dereddening')
        else:
            end = True
            print('Not valid value for Lutz or Draine dereddening plot!')
            break
        
        # Create the blackbody regions
        medium_bb_stellar = sirtipy.region()
        medium_bb_ir      = sirtipy.region()
            
    # Create the synchrotron region
    medium_sync = sirtipy.region()
    
    # Initialize/Re-initialize values
    spec_sync, tau_sync, locations_sync = 0, 0, 0
    
    # Create a tuple that contains all of the parameters we call the synchrotron functions with
    func_params_sync = (i, s_radio, s_ir, gmin, j, k, cutoff_freq,)

    # Add emission functions for both synchrotrons
    medium_sync.add_emission_func(j_synchrotron, func_params_sync)
    
    # Calculate the radiative transfer to the final distance
    spec_sync, tau_sync, locations_sync = sirtipy.radiative_transfer(medium_sync, inspec_empty_sync, s, ds)
        
    # Find the line of sight for given distances
    los_plot_sync_radio  = np.searchsorted(locations_sync, distances_sync_radio)
    if sim_sync_ir == True:
        los_plot_sync_ir = np.searchsorted(locations_sync, distances_sync_ir)
        
    # Convert from intensity to flux
    spec_flux_sync = inten_to_flux(spec_sync.intensities()[los_plot_sync_radio], r_disk)
    if sim_sync_ir == True:
        spec_flux_sync_ir = inten_to_flux(spec_sync.intensities()[los_plot_sync_ir[0]], r_disk)
    
    # Run simulations for Star Blackbody, IR Blackbody, and IR Synchrotron once
    if counter == 1:
        # Add absorption and emission functions for both blackbodies
        medium_bb_stellar.add_absorption_func(alpha_thermal, (alpha_bb_stellar,))
        medium_bb_stellar.add_emission_func(j_thermal, (medium_bb_stellar_temp, alpha_thermal, (alpha_bb_stellar,)))
        
        if sim_bb_ir == True:
            medium_bb_ir.add_absorption_func(alpha_thermal, (alpha_bb_ir,))
            medium_bb_ir.add_emission_func(j_thermal, (medium_bb_ir_temp, alpha_thermal, (alpha_bb_ir,)))
        
        # Calculate the radiative transfer to the final distance
        spec_bb_stellar, tau_bb_stellar, locations_bb_stellar = sirtipy.radiative_transfer(medium_bb_stellar, inspec_empty_bb_stellar, s, ds)
        if sim_bb_ir == True:
            spec_bb_ir, tau_bb_ir, locations_bb_ir = sirtipy.radiative_transfer(medium_bb_ir, inspec_empty_bb_ir, s, ds)
        
        # Find the line of sight for given distances
        los_plot_bb_stellar = np.searchsorted(locations_bb_stellar, distances_bb_stellar)
        if sim_bb_ir == True:
            los_plot_bb_ir  = np.searchsorted(locations_bb_ir, distances_bb_ir)
  
        # Convert from intensity to flux
        spec_flux_bb_stellar = inten_to_flux(spec_bb_stellar.intensities()[los_plot_bb_stellar[0]], radius_bb_stellar)
        if sim_bb_ir == True:
            spec_flux_bb_ir  = inten_to_flux(spec_bb_ir.intensities()[los_plot_bb_ir[0]], medium_bb_ir_rad)
        
    # Fit line of sight for gbi data and second synchrotron if needed
    if counter == 1:
        draine_freq_window = []
        lutz_freq_window   = []
        gbi_freq_window    = np.where(np.isclose(freqaxis.frequencies, gbi_freq_scaled[0], atol=crit_freq[0]))[0]
        
        # Initialize for finding best fit
        mse_combined = 1000.

        for m in range(len(draine_freq_scaled)):
            draine_freq_window.append(np.where(np.isclose(freqaxis.frequencies, draine_freq_scaled[m], atol=draine_error_freq[m]))[0][0])            
            lutz_freq_window.append(np.where(np.isclose(freqaxis.frequencies, lutz_freq_scaled[m], atol=lutz_error_freq[m]))[0][0])
    
    gbi_flux_window = np.unique(np.where(np.isclose(spec_flux_sync[:, gbi_freq_window], gbi_flux[0], atol=crit_flux[0]))[0])
    
    if (cutoff_freq == False):
        los_sync = gbi_flux_window
        spec_flux_sync_radio = spec_flux_sync[los_sync]
    else:
        los_sync = np.where(np.isclose(spec_flux_sync[gbi_flux_window, cutoff_index], spec_flux_sync_ir[cutoff_index+1], atol=spec_flux_sync_ir[cutoff_index+1] * .05))[0]
        spec_flux_sync_radio = spec_flux_sync[gbi_flux_window[los_sync]]
        
    # Proceed with calculations and plots if line of sight found
    if len(los_sync) != 0:
        print('Line of sight found for given parameters!')
        full_spec_combo[0].append(title + f'B = {i:.3E}, gmax = {j:.3E}, ne = {k:.3E}')

        # Sum of desired components
        spec_combined = []
        if (cutoff_freq   == False):
            if (sim_bb_ir == True):
                for x in range(len(los_sync)):
                    spec_combined.append(spec_flux_bb_ir + spec_flux_bb_stellar + spec_flux_sync_radio[x])
            else:
                for x in range(len(los_sync)):
                    spec_combined.append(spec_flux_bb_stellar + spec_flux_sync_radio[x])
        else:
            spec_flux_sync_combined = []
            for x in range(len(los_sync)):
                spec_flux_sync_combined.append(list(itertools.chain(spec_flux_sync_radio[x][freqaxis.frequencies < cutoff_freq], spec_flux_sync_ir[freqaxis.frequencies > cutoff_freq])))
                if (sim_bb_ir == True):
                    spec_combined.append(spec_flux_bb_ir + spec_flux_bb_stellar + spec_flux_sync_combined[x])
                else:
                    spec_combined.append(spec_flux_bb_stellar + spec_flux_sync_combined[x])
        
        # Calculate Draine mean squared error
        if plot == "Draine":
            flux_gbi_true    = [gbidata[0][1], gbidata[1][1]]
            flux_gbi_pred    = []
            flux_gbi_mse     = []
            flux_full_mse    = []
            flux_draine_true = draine_flux
            flux_draine_pred = []
            flux_draine_mse  = []
            
            for y in range(len(spec_combined)):
                flux_gbi_pred.append([spec_combined[y][gbi_freq_window[0]], spec_combined[y][gbi_freq_window[1]]])
                flux_gbi_mse.append(mean_squared_error(flux_gbi_true, flux_gbi_pred[y]))
                flux_draine_pred.append(spec_combined[y][draine_freq_window])
                flux_draine_mse.append(mean_squared_error(flux_draine_true, flux_draine_pred[y]))
                flux_full_mse.append(flux_gbi_mse[y] + flux_draine_mse[y])

            los_best_fit = np.where(flux_full_mse == np.min(flux_full_mse))[0]
            full_spec_combo[1].append(spec_combined[los_best_fit[0]]) 
            full_spec_combo[2].append([flux_gbi_mse[los_best_fit[0]], flux_draine_mse[los_best_fit[0]]])
            full_spec_combo[3].append(flux_full_mse[los_best_fit[0]])
        
        # Calculate Lutz mean squared error
        elif plot == "Lutz":
            flux_gbi_true  = [gbidata[0][1], gbidata[1][1]]
            flux_gbi_pred  = []
            flux_gbi_mse   = []
            flux_full_mse  = []
            flux_lutz_true = lutz_flux
            flux_lutz_pred = []
            flux_lutz_mse  = []

            for y in range(len(spec_combined)):
                flux_gbi_pred.append([spec_combined[y][gbi_freq_window[0]], spec_combined[y][gbi_freq_window[1]]])
                flux_gbi_mse.append(mean_squared_error(flux_gbi_true, flux_gbi_pred[y]))
                flux_lutz_pred.append(spec_combined[y][lutz_freq_window])
                flux_lutz_mse.append(mean_squared_error(flux_lutz_true, flux_lutz_pred[y]))
                flux_full_mse.append(flux_gbi_mse[y] + flux_lutz_mse[y])

            los_best_fit = np.where(flux_full_mse == np.min(flux_full_mse))[0]
            full_spec_combo[1].append(spec_combined[los_best_fit[0]]) 
            full_spec_combo[2].append([flux_gbi_mse[los_best_fit[0]], flux_lutz_mse[los_best_fit[0]]])
            full_spec_combo[3].append(flux_full_mse[los_best_fit[0]])
            
        if full_spec_combo[3][-1] < mse_combined:
            mse_combined = full_spec_combo[3][-1]
            print('New best fit found')
            print(f'Sum of residuals: {mse_combined}')
            
            # Plot everything
            plt.figure(figsize=(9,6))
            plt.rcParams.update({'font.size': 15})

            # Plot synchrotrons
            if sim_sync_ir == True:
                plot_lesser_cutoff_freq  = np.where(freqaxis.frequencies < cutoff_freq)[0]
                plot_greater_cutoff_freq = np.where(freqaxis.frequencies > cutoff_freq)[0]
                plt.plot(freqaxis.frequencies[plot_lesser_cutoff_freq] / scale_freq, spec_flux_sync_radio[los_best_fit[0]][freqaxis.frequencies < cutoff_freq], color='blue',
                         label=f'Radio Synchrotron, s={locations_sync[los_plot_sync_radio[los_best_fit[0]]]:.3E} cm\nFlux $\\propto \\nu^{{-{s_radio}}}$ for $\\nu <$ {cutoff_freq/1e9:.0f} GHz')
                plt.plot(freqaxis.frequencies[plot_greater_cutoff_freq] / scale_freq, spec_flux_sync_ir[freqaxis.frequencies > cutoff_freq], color='orange',
                         label=f'IR Synchrotron, s={distances_sync_ir[0]:.3E} cm\nFlux $\\propto \\nu^{{-{s_ir}}}$ for $\\nu >$ {cutoff_freq/1e9:.0f} GHz')
            else:
                plt.plot(freqaxis.frequencies / scale_freq, spec_flux_sync_radio[los_best_fit[0]], color='blue',
                         label=f'Radio Synchrotron, s={locations_sync[los_plot_sync_radio[los_best_fit[0]]]:.3E} cm\nFlux $\\propto \\nu^{{{s_radio}}}$')

            # Plot blackbodies
            plt.plot(freqaxis.frequencies / scale_freq, spec_flux_bb_stellar, ls='dashed', color='green', label=f'Star, T = {medium_bb_stellar_temp} K, R = {radius_bb_stellar / r_sun} R$_\\odot$')
            if sim_bb_ir == True:
                plt.plot(freqaxis.frequencies / scale_freq, spec_flux_bb_ir, ls='dashdot', 
                         color='red', label=f'Blackbody, T = {medium_bb_ir_temp} K, R = {medium_bb_ir_rad / r_sun} R$_\\odot$',)

            # Plot combined
            plt.plot(freqaxis.frequencies / scale_freq, spec_combined[los_best_fit[0]], ls='solid', c='black', label='Sum of Components')

            # Plot data and errorbars for GBI, Lutz, and Draine data
            plt.errorbar(gbi_freq, gbi_flux, xerr=None, yerr=gbi_error_flux, 
                         ls='none', color='purple', capsize=3, label='GBI Data')

            # Make title, crop with limits, use log scale, label axes, and move legend
            if (plot == 'Lutz'):
                plt.title(title + f'B = {i:.3E}, gmax = {j:.3E}, ne = {k:.3E}')
                plt.errorbar(lutz_freq, lutz_flux, xerr=lutz_error_freq/1e8, color='brown', 
                             yerr=lutz_error_flux, ls='none', capsize=3, label=label)
            elif (plot == 'Draine'):
                plt.title(title + f'B = {i:.3E}, gmax = {j:.3E}, ne = {k:.3E}')
                plt.errorbar(draine_freq, draine_flux, xerr=draine_error_freq/1e8, color='brown', 
                             yerr=draine_error_flux, ls='none', capsize=3, label=label)
            else:
                plt.title(title + f'B = {i:.3E}, gmax = {j:.3E}, ne = {k:.3E}')

            plt.xlim(1, 4e5)
            plt.ylim(4e0, 3e2)
            plt.xscale('log')
            plt.yscale('log')
            plt.tick_params(which='both', length=5, width=2)
            plt.xticks()
            plt.yticks()
            plt.xlabel('$\\nu$ (GHz)')
            plt.ylabel('flux density (mJy)')
            plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')

            # Save figure with unique name
            if save_fig == True:
                plot_name = 'radio_sync_star_bb'
                if sim_sync_ir == True:
                    plot_name  += f'_ir_sync_{cutoff_freq:.3E}'
                if sim_bb_ir   == True:
                    plot_name  += f'_ir_bb_{medium_bb_ir_temp:.3E}'
                if dereddening_lutz     == True:
                    plot_name  += '_lutz_dered'
                elif dereddening_draine == True:
                    plot_name  += '_draine_dered'

                # Make directory for saved plots
                os.makedirs(f'plots/{plot_name}-extreme', exist_ok=True)

                # Save figure
                plt.savefig(f'./plots/{plot_name}-extreme/B{i:.3E}-gmax{j:.3E}-ne{k:.3E}-{plot_name}.png', bbox_inches='tight')

            # Print plots after each simulation
            if plot_individual == True:
                plt.show()
            
            # Close figure to avoid plotting
            else:
                plt.close(fig=None)
        
        else:
            print('No new best fit found.')
            print(f'Current sum of residuals: {full_spec_combo[3][-1]}')
            print(f'Best sum of residuals: {mse_combined}')
    
    else:
        print('No line of sight found for given parameters!')
    
    # Print time elapsed
    if timer == True:
        iteration_time = time.perf_counter() - time_iteration
        execution_time = time.perf_counter() - start
        print(f'Complete B = {i:.3E}, gmax = {j:.3E}, ne = {k:.3E}')
        print(f'Iteration {counter} finished in {iteration_time:.3} seconds')
        print(f'Program has been running for {execution_time / 60.:.3} minutes\n')
        
# Print total execution time and average iteration time
# Only for curiosity sake
if (timer == True) & (end == False):
    print(f'Program ran for {execution_time / 60.:.3} minutes')
    print(f'Program took an average of {execution_time / iterations:.3} seconds per iteration')

Iteration 1 of 8000
Begin B = 1.000E+03, gmax = 1.000E+04, ne = 1.000E+03
IR synchrotron frequency cutoff 31000e9 GHz
No IR Blackbody
Draine Dereddening
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.000E+04, ne = 1.000E+03
Iteration 1 finished in 4.1 seconds
Program has been running for 0.0684 minutes

Iteration 2 of 8000
Begin B = 1.000E+03, gmax = 1.000E+04, ne = 1.438E+03
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.000E+04, ne = 1.438E+03
Iteration 2 finished in 0.657 seconds
Program has been running for 0.0793 minutes

Iteration 3 of 8000
Begin B = 1.000E+03, gmax = 1.000E+04, ne = 2.069E+03
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.000E+04, ne = 2.069E+03
Iteration 3 finished in 0.649 seconds
Program has been running for 0.0901 minutes

Iteration 4 of 8000
Begin B = 1.000E+03, gmax = 1.000E+04, ne = 2.976E+03
No line of sight found for given parameters!
Complete B = 1.000E+03, gma

No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.438E+04, ne = 5.456E+04
Iteration 32 finished in 0.607 seconds
Program has been running for 0.383 minutes

Iteration 33 of 8000
Begin B = 1.000E+03, gmax = 1.438E+04, ne = 7.848E+04
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.438E+04, ne = 7.848E+04
Iteration 33 finished in 0.598 seconds
Program has been running for 0.393 minutes

Iteration 34 of 8000
Begin B = 1.000E+03, gmax = 1.438E+04, ne = 1.129E+05
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.438E+04, ne = 1.129E+05
Iteration 34 finished in 0.591 seconds
Program has been running for 0.402 minutes

Iteration 35 of 8000
Begin B = 1.000E+03, gmax = 1.438E+04, ne = 1.624E+05
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 1.438E+04, ne = 1.624E+05
Iteration 35 finished in 0.588 seconds
Program has been running for 0.412 minutes

Iteration 36 of 8000
Begin B = 1.00

No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 2.976E+04, ne = 2.976E+03
Iteration 64 finished in 0.633 seconds
Program has been running for 0.705 minutes

Iteration 65 of 8000
Begin B = 1.000E+03, gmax = 2.976E+04, ne = 4.281E+03
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 2.976E+04, ne = 4.281E+03
Iteration 65 finished in 0.617 seconds
Program has been running for 0.715 minutes

Iteration 66 of 8000
Begin B = 1.000E+03, gmax = 2.976E+04, ne = 6.158E+03
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 2.976E+04, ne = 6.158E+03
Iteration 66 finished in 0.625 seconds
Program has been running for 0.726 minutes

Iteration 67 of 8000
Begin B = 1.000E+03, gmax = 2.976E+04, ne = 8.859E+03
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 2.976E+04, ne = 8.859E+03
Iteration 67 finished in 0.603 seconds
Program has been running for 0.736 minutes

Iteration 68 of 8000
Begin B = 1.00

No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 4.281E+04, ne = 2.336E+05
Iteration 96 finished in 0.598 seconds
Program has been running for 1.03 minutes

Iteration 97 of 8000
Begin B = 1.000E+03, gmax = 4.281E+04, ne = 3.360E+05
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 4.281E+04, ne = 3.360E+05
Iteration 97 finished in 0.597 seconds
Program has been running for 1.04 minutes

Iteration 98 of 8000
Begin B = 1.000E+03, gmax = 4.281E+04, ne = 4.833E+05
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 4.281E+04, ne = 4.833E+05
Iteration 98 finished in 0.6 seconds
Program has been running for 1.05 minutes

Iteration 99 of 8000
Begin B = 1.000E+03, gmax = 4.281E+04, ne = 6.952E+05
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 4.281E+04, ne = 6.952E+05
Iteration 99 finished in 0.602 seconds
Program has been running for 1.06 minutes

Iteration 100 of 8000
Begin B = 1.000E+03

No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 8.859E+04, ne = 1.274E+04
Iteration 128 finished in 0.597 seconds
Program has been running for 1.35 minutes

Iteration 129 of 8000
Begin B = 1.000E+03, gmax = 8.859E+04, ne = 1.833E+04
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 8.859E+04, ne = 1.833E+04
Iteration 129 finished in 0.604 seconds
Program has been running for 1.36 minutes

Iteration 130 of 8000
Begin B = 1.000E+03, gmax = 8.859E+04, ne = 2.637E+04
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 8.859E+04, ne = 2.637E+04
Iteration 130 finished in 0.616 seconds
Program has been running for 1.38 minutes

Iteration 131 of 8000
Begin B = 1.000E+03, gmax = 8.859E+04, ne = 3.793E+04
No line of sight found for given parameters!
Complete B = 1.000E+03, gmax = 8.859E+04, ne = 3.793E+04
Iteration 131 finished in 0.605 seconds
Program has been running for 1.39 minutes

Iteration 132 of 8000
Begin B = 

...

No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 7.848E+05, ne = 4.833E+05
Iteration 7858 finished in 0.597 seconds
Program has been running for 82.4 minutes

Iteration 7859 of 8000
Begin B = 1.000E+06, gmax = 7.848E+05, ne = 6.952E+05
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 7.848E+05, ne = 6.952E+05
Iteration 7859 finished in 0.597 seconds
Program has been running for 82.5 minutes

Iteration 7860 of 8000
Begin B = 1.000E+06, gmax = 7.848E+05, ne = 1.000E+06
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 7.848E+05, ne = 1.000E+06
Iteration 7860 finished in 0.6 seconds
Program has been running for 82.5 minutes

Iteration 7861 of 8000
Begin B = 1.000E+06, gmax = 1.129E+06, ne = 1.000E+03
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.129E+06, ne = 1.000E+03
Iteration 7861 finished in 0.6 seconds
Program has been running for 82.5 minutes

Iteration 7862 of 8000
Begin 

No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.624E+06, ne = 2.637E+04
Iteration 7890 finished in 0.626 seconds
Program has been running for 82.8 minutes

Iteration 7891 of 8000
Begin B = 1.000E+06, gmax = 1.624E+06, ne = 3.793E+04
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.624E+06, ne = 3.793E+04
Iteration 7891 finished in 0.596 seconds
Program has been running for 82.8 minutes

Iteration 7892 of 8000
Begin B = 1.000E+06, gmax = 1.624E+06, ne = 5.456E+04
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.624E+06, ne = 5.456E+04
Iteration 7892 finished in 0.601 seconds
Program has been running for 82.8 minutes

Iteration 7893 of 8000
Begin B = 1.000E+06, gmax = 1.624E+06, ne = 7.848E+04
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.624E+06, ne = 7.848E+04
Iteration 7893 finished in 0.594 seconds
Program has been running for 82.8 minutes

Iteration 7894 of 8000
Be

No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 3.360E+06, ne = 1.438E+03
Iteration 7922 finished in 0.596 seconds
Program has been running for 83.1 minutes

Iteration 7923 of 8000
Begin B = 1.000E+06, gmax = 3.360E+06, ne = 2.069E+03
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 3.360E+06, ne = 2.069E+03
Iteration 7923 finished in 0.596 seconds
Program has been running for 83.1 minutes

Iteration 7924 of 8000
Begin B = 1.000E+06, gmax = 3.360E+06, ne = 2.976E+03
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 3.360E+06, ne = 2.976E+03
Iteration 7924 finished in 0.597 seconds
Program has been running for 83.1 minutes

Iteration 7925 of 8000
Begin B = 1.000E+06, gmax = 3.360E+06, ne = 4.281E+03
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 3.360E+06, ne = 4.281E+03
Iteration 7925 finished in 0.594 seconds
Program has been running for 83.1 minutes

Iteration 7926 of 8000
Be

No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 4.833E+06, ne = 1.129E+05
Iteration 7954 finished in 0.597 seconds
Program has been running for 83.4 minutes

Iteration 7955 of 8000
Begin B = 1.000E+06, gmax = 4.833E+06, ne = 1.624E+05
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 4.833E+06, ne = 1.624E+05
Iteration 7955 finished in 0.597 seconds
Program has been running for 83.4 minutes

Iteration 7956 of 8000
Begin B = 1.000E+06, gmax = 4.833E+06, ne = 2.336E+05
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 4.833E+06, ne = 2.336E+05
Iteration 7956 finished in 0.595 seconds
Program has been running for 83.4 minutes

Iteration 7957 of 8000
Begin B = 1.000E+06, gmax = 4.833E+06, ne = 3.360E+05
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 4.833E+06, ne = 3.360E+05
Iteration 7957 finished in 0.596 seconds
Program has been running for 83.4 minutes

Iteration 7958 of 8000
Be

No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.000E+07, ne = 6.158E+03
Iteration 7986 finished in 0.593 seconds
Program has been running for 83.7 minutes

Iteration 7987 of 8000
Begin B = 1.000E+06, gmax = 1.000E+07, ne = 8.859E+03
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.000E+07, ne = 8.859E+03
Iteration 7987 finished in 0.593 seconds
Program has been running for 83.7 minutes

Iteration 7988 of 8000
Begin B = 1.000E+06, gmax = 1.000E+07, ne = 1.274E+04
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.000E+07, ne = 1.274E+04
Iteration 7988 finished in 0.596 seconds
Program has been running for 83.7 minutes

Iteration 7989 of 8000
Begin B = 1.000E+06, gmax = 1.000E+07, ne = 1.833E+04
No line of sight found for given parameters!
Complete B = 1.000E+06, gmax = 1.000E+07, ne = 1.833E+04
Iteration 7989 finished in 0.592 seconds
Program has been running for 83.7 minutes

Iteration 7990 of 8000
Be

In [1]:
# Find the best fit parameters TOP RIGHT
# full_spec_combo_top_right_extreme = full_spec_combo
# param_best_fit_top_right_extreme = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_top_right_extreme = np.min(full_spec_combo[3])
# best_fit_name_top_right_extreme = full_spec_combo[0][param_best_fit_top_right_extreme]
# print('param_best_fit_index top right plot', param_best_fit_top_right_extreme)
# print('Mean Squared Error top right plot', mse_top_right_extreme)
# print('Best fit parameters top right plot', best_fit_name_top_right_extreme)

In [499]:
# # Find the best fit parameters MIDDLE RIGHT
# full_spec_combo_middle_right_extreme = full_spec_combo
# param_best_fit_middle_right_extreme = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_middle_right_extreme = np.min(full_spec_combo[3])
# best_fit_name_middle_right_extreme = full_spec_combo[0][param_best_fit_middle_right_extreme]
# print('param_best_fit_index middle right plot', param_best_fit_middle_right_extreme)
# print('Mean Squared Error middle right plot', mse_middle_right_extreme)
# print('Best fit parameters middle right plot', best_fit_name_middle_right_extreme)
# np.savetxt('./best_fits/ir_sync_cutoff_31000GHz_spectra.txt', full_spec_combo_middle_right_extreme[1])

param_best_fit_index middle right plot 171
Mean Squared Error middle right plot 8.38611013056878
Best fit parameters middle right plot GRS 1915+105 (Draine Dereddening)
B = 5.456E+04, gmax = 5.456E+05, ne = 1.438E+03


In [465]:
# # Find the best fit parameters BOTTOM RIGHT
# full_spec_combo_bottom_right_extreme = full_spec_combo
# param_best_fit_bottom_right_extreme = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_bottom_right_extreme = np.min(full_spec_combo[3])
# best_fit_name_bottom_right_extreme = full_spec_combo[0][param_best_fit_bottom_right_extreme]
# print('param_best_fit_index bottom right plot', param_best_fit_bottom_right_extreme)
# print('Mean Squared Error bottom right plot', mse_bottom_right_extreme)
# print('Best fit parameters bottom right plot', best_fit_name_bottom_right_extreme)
np.savetxt('./best_fits/ir_sync_cutoff_2000GHz_ir_bb_500K_spectra.txt', full_spec_combo_bottom_right_extreme[1])

In [463]:
# # Find the best fit parameters TOP LEFT
# full_spec_combo_top_left_extreme = full_spec_combo
# param_best_fit_top_left_extreme = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_top_left_extreme = np.min(full_spec_combo[3])
# best_fit_name_top_left_extreme = full_spec_combo[0][param_best_fit_top_left_extreme]
# print('param_best_fit_index top left plot', param_best_fit_top_left_extreme)
# print('Mean Squared Error top left plot', mse_top_left_extreme)
# print('Best fit parameters top left plot', best_fit_name_top_left_extreme)
np.savetxt('./best_fits/ir_bb_1100K_spectra.txt', full_spec_combo_top_left_extreme[1])

In [461]:
# Find the best fit parameters MIDDLE LEFT
# full_spec_combo_middle_left_extreme = full_spec_combo
# param_best_fit_middle_left_extreme = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_middle_left_extreme = np.min(full_spec_combo[3])
# best_fit_name_middle_left_extreme = full_spec_combo[0][param_best_fit_middle_left_extreme]
# print('param_best_fit_index middle left plot', param_best_fit_middle_left_extreme)
# print('Mean Squared Error middle left plot', mse_middle_left_extreme)
# print('Best fit parameters middle left plot', best_fit_name_middle_left_extreme)
np.savetxt('./best_fits/ir_sync_cutoff_9000GHz_ir_bb_950K_spectra.txt', full_spec_combo_middle_left_extreme[1])

In [459]:
# # Find the best fit parameters BOTTOM LEFT
# full_spec_combo_bottom_left_extreme = full_spec_combo
# param_best_fit_bottom_left_extreme = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_bottom_left_extreme = np.min(full_spec_combo[3])
# best_fit_name_bottom_left_extreme = full_spec_combo[0][param_best_fit_bottom_left_extreme]
# print('param_best_fit_index bottom left plot', param_best_fit_bottom_left_extreme)
# print('Mean Squared Error bottom left plot', mse_bottom_left_extreme)
# print('Best fit parameters bottom left plot', best_fit_name_bottom_left_extreme)
np.savetxt('./best_fits/ir_sync_cutoff_2000GHz_ir_bb_900K_residuals.txt', full_spec_combo_bottom_left_extreme[3])

In [170]:
# # Find the best fit parameters TOP RIGHT
# param_best_fit_top_right = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_top_right = np.min(full_spec_combo[3])
# best_fit_name_top_right = full_spec_combo[0][param_best_fit_top_right]
print('param_best_fit_index top right plot', param_best_fit_top_right)
print('Mean Squared Error top right plot', mse_top_right)
print('Best fit parameters top right plot', best_fit_name_top_right)

param_best_fit_index top right plot 47
Mean Squared Error top right plot 11.516761475725502
Best fit parameters top right plot GRS 1915+105 (Draine Dereddening)
B = 1.000E+03, gmax = 4.642E+06, ne = 4.642E+04


In [243]:
# # Find the best fit parameters MIDDLE RIGHT
# param_best_fit_middle_right = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_middle_right = np.min(full_spec_combo[3])
# best_fit_name_middle_right = full_spec_combo[0][param_best_fit_middle_right]
print('param_best_fit_index middle right plot', param_best_fit_middle_right)
print('Mean Squared Error middle right plot', mse_middle_right)
print('Best fit parameters middle right plot', best_fit_name_middle_right)

param_best_fit_index middle right plot 9
Mean Squared Error middle right plot 8.301143650159796
Best fit parameters middle right plot GRS 1915+105 (Draine Dereddening)
B = 2.154E+03, gmax = 1.000E+07, ne = 4.642E+05


In [245]:
# Find the best fit parameters BOTTOM RIGHT
# param_best_fit_bottom_right = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_bottom_right = np.min(full_spec_combo[3])
# best_fit_name_bottom_right = full_spec_combo[0][param_best_fit_bottom_right]
print('param_best_fit_index bottom right plot', param_best_fit_bottom_right)
print('Mean Squared Error bottom right plot', mse_bottom_right)
print('Best fit parameters bottom right plot', best_fit_name_bottom_right)

param_best_fit_index bottom right plot 16
Mean Squared Error bottom right plot 15.079558009489244
Best fit parameters bottom right plot GRS 1915+105 (Draine Dereddening)
B = 2.154E+04, gmax = 1.000E+06, ne = 1.000E+03


In [168]:
# # Find the best fit parameters TOP LEFT
# param_best_fit_top_left = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_top_left = np.min(full_spec_combo[3])
# best_fit_name_top_left = full_spec_combo[0][param_best_fit_top_left]
print('param_best_fit_index top left plot', param_best_fit_top_left)
print('Mean Squared Error top left plot', mse_top_left)
print('Best fit parameters top left plot', best_fit_name_top_left)

param_best_fit_index top left plot 205
Mean Squared Error top left plot 53.03046332804147
Best fit parameters top left plot GRS 1915+105 (Lutz Dereddening)
B = 1.000E+04, gmax = 1.000E+04, ne = 4.642E+04


In [247]:
# Find the best fit parameters MIDDLE LEFT
# param_best_fit_middle_left = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_middle_left = np.min(full_spec_combo[3])
# best_fit_name_middle_left = full_spec_combo[0][param_best_fit_middle_left]
print('param_best_fit_index middle left plot', param_best_fit_middle_left)
print('Mean Squared Error middle left plot', mse_middle_left)
print('Best fit parameters middle left plot', best_fit_name_middle_left)

param_best_fit_index middle left plot 15
Mean Squared Error middle left plot 38.7631547292266
Best fit parameters middle left plot GRS 1915+105 (Lutz Dereddening)
B = 4.642E+03, gmax = 4.642E+05, ne = 4.642E+04


In [249]:
# Find the best fit parameters BOTTOM LEFT
# param_best_fit_bottom_left = np.where(full_spec_combo[3] == np.min(full_spec_combo[3]))[0][0]
# mse_bottom_left = np.min(full_spec_combo[3])
# best_fit_name_bottom_left = full_spec_combo[0][param_best_fit_bottom_left]
print('param_best_fit_index bottom left plot', param_best_fit_bottom_left)
print('Mean Squared Error bottom left plot', mse_bottom_left)
print('Best fit parameters bottom left plot', best_fit_name_bottom_left)

param_best_fit_index bottom left plot 16
Mean Squared Error bottom left plot 20.7048730689961
Best fit parameters bottom left plot GRS 1915+105 (Lutz Dereddening)
B = 2.154E+04, gmax = 1.000E+06, ne = 1.000E+03
